In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20,256),
    nn.ReLU(),
    nn.Linear(256,10))
X = torch.rand(2, 20)
net(X)

tensor([[-1.9299e-01,  2.6591e-02, -1.8691e-04,  6.2071e-02, -1.0477e-01,
          2.1494e-01, -1.5033e-02,  1.3259e-01,  1.6497e-03, -2.1067e-01],
        [-1.9454e-01, -3.6067e-02,  5.8015e-02,  1.3466e-01, -5.3942e-02,
          9.2310e-02,  8.6950e-02,  2.3103e-01,  7.2298e-02, -2.7761e-01]],
       grad_fn=<AddmmBackward0>)

#### 自定义块
从零开始编写一个块。 它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。 注意，下面的MLP类继承了表示块的类。 我们的实现只需要提供我们自己的构造函数（Python中的__init__函数）和前向传播函数。

In [5]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [11]:
net = MLP()
net(X)

tensor([[-0.1542,  0.0282, -0.1982,  0.0508, -0.0958,  0.0309,  0.1501, -0.2069,
         -0.1491, -0.0127],
        [-0.1597,  0.0608, -0.2833,  0.0565, -0.0515,  0.0341,  0.1243, -0.1932,
         -0.1912, -0.0879]], grad_fn=<AddmmBackward0>)

在这个例子中，我们通过实例化nn.Sequential来构建我们的模型， 层的执行顺序是作为参数传递的。 简而言之，nn.Sequential定义了一种特殊的Module， 即在PyTorch中表示一个块的类， 它维护了一个由Module组成的有序列表。 注意，两个全连接层都是Linear类的实例， Linear类本身就是Module的子类。 另外，到目前为止，我们一直在通过net(X)调用我们的模型来获得模型的输出。 这实际上是net.__call__(X)的简写。 这个前向传播函数非常简单： 它将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入。

#### 顺序块
现在我们可以更仔细地看看Sequential类是如何工作的， 回想一下Sequential的设计是为了把其他模块串起来。 为了构建我们自己的简化的MySequential， 我们只需要定义两个关键函数：

一种将块逐个追加到列表中的函数；

一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

下面的MySequential类提供了与默认Sequential类相同的功能。

In [12]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

### 在前向传播过程中执行代码

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
         # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2 
        return X.sum()

In [7]:
net = FixedHiddenMLP()
print(net[0].weight.data[:])

TypeError: 'FixedHiddenMLP' object is not subscriptable

### 我们可以混合搭配各种组合块的方法。 在下面的例子中，我们以一些想到的方法嵌套块。

In [6]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.2133, grad_fn=<SumBackward0>)

### 练习 ：实现一个块，它以两个块为参数，例如net1和net2，并返回前向传播中两个网络的串联输出。这也被称为平行块。

In [22]:
import torch
from torch import nn

class mynet(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.__modules[str(idx)] = module
        
    def forward(self, X):
        return torch.concat((self._modules['0'](X),self._modules['1'](X) ), 1)

In [16]:
X = torch.rand(2, 20)
n1 = nn.Linear(20, 256)
n2 = nn.Linear(20, 128)
p = MyParallel(n1,n2)
print(p(X).shape, p(X))
print(p)

NameError: name 'MyParallel' is not defined